In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends
import pickle

import pyExSi

import LadiskDAQ

%matplotlib qt

In [2]:
acq1 = LadiskDAQ.NIAcquisition("TestInputTask", acquisition_name="NI_task")

acq2 = LadiskDAQ.SerialAcquisition(port="COM3", baudrate=250000, timeout=5,
                                  byte_sequence=(("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb",
                                  end_bytes=b"\n",
                                  pretest_time= 60.,
                                  sample_rate=500.,
                                  acquisition_name="Arduino_1"
                                  )

acq3 = LadiskDAQ.SerialAcquisition(port="COM8", baudrate=250000, timeout=5,
                                  byte_sequence=(("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb", 
                                  end_bytes=b"\n",
                                  pretest_time= None,
                                  sample_rate=200,
                                  acquisition_name="Arduino_2"
                                  )


In [3]:
acq3.acquisition_name

'Arduino_2'

In [4]:
acq1.sample_rate, acq2.sample_rate, acq3.sample_rate

(25600.0, 500.0, 200)

In [5]:
# create DAQ object:
ldaq = LadiskDAQ.Core(acquisitions=[acq2, acq1, acq3])
ldaq.set_global_trigger(
     trigger_source = "Arduino_1", 
     trigger_channel = 1, 
     trigger_level = 100.,
)

In [6]:
ldaq.run(run_time=10.0)

+--------+-------------------------------------------------+
| HOTKEY |                   DESCRIPTION                   |
+--------+-------------------------------------------------+
|   s    | Start the measurement manually (ignore trigger) |
+--------+-------------------------------------------------+
|   q    |              Stop the measurement               |
+--------+-------------------------------------------------+
|   f    |            Freeze the plot (toggle)             |
+--------+-------------------------------------------------+
	Waiting for trigger...triggered.
	Recording...Measurement finished.


In [9]:
measurement_dict = ldaq.get_measurement_dict(last_N_seconds=3.)

In [10]:
fig, axs = plt.subplots(3, sharex=True)
for idx, source in enumerate(list(measurement_dict.keys())):
    meas = measurement_dict[source]
    axs[idx].plot( meas["time"], meas["data"] )


In [11]:
ldaq.save_measurement("ldaq", root=r"C:\LADISK\packages\test_meas")

In [12]:
LadiskDAQ.load_measurement("20230404_124611_ldaq.pkl", r"C:\LADISK\packages\test_meas")

{'Arduino_1': {'data': array([[  0., 447.],
         [  0., 443.],
         [  0., 441.],
         ...,
         [  0., 202.],
         [  0., 203.],
         [  0., 205.]]),
  'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 2.532e+00, 2.534e+00,
         2.536e+00]),
  'channel_names': ['channel 1', 'channel 2'],
  'sample_rate': 500.0},
 'NI_task': {'data': array([[1.56550000e-04, 1.65001896e+00],
         [4.41835000e-04, 1.65033407e+00],
         [5.89525000e-05, 1.65000395e+00],
         ...,
         [9.98841737e-01, 2.56063322e-01],
         [9.98916812e-01, 2.56205872e-01],
         [9.98961857e-01, 2.56839845e-01]]),
  'time': array([0.00000000e+00, 3.90625000e-05, 7.81250000e-05, ...,
         2.47671875e+00, 2.47675781e+00, 2.47679687e+00]),
  'channel_names': ['Voltage', 'Voltage_0'],
  'sample_rate': 25600.0},
 'Arduino_2': {'data': array([[  0., 436.],
         [  0., 432.],
         [  0., 428.],
         ...,
         [204., 192.],
         [204., 196.],
         [